In [ ]:
from __future__ import annotations

from requests.adapters import HTTPAdapter
import requests
from urllib3.util.retry import Retry

class HttpMethod:
    def __init__(self):
        self.session = self._requests_retry_session()

    def _requests_retry_session(retries=5, backoff_factor=0.3,
                                status_forcelist=(500, 502, 504), session=None):
        
        with requests.Session() as s:
        
            retry = Retry(total=retries, read=retries, connect=retries,
                            backoff_factor=backoff_factor,
                            status_forcelist=status_forcelist)
            adapter = HTTPAdapter(max_retries=retry)
            s.mount('http://', adapter)
            s.mount('https://', adapter)
            return s
    
    def get(self, path, timeout=3, **kwargs):
        try:
            uri = self.base_url + path
            return self.session.get(url=uri, params=kwargs, timeout=timeout).\
                json()
        except Exception as x:
            print("It failed", x.__class__.__name__)
            return None

    
class BithumbHttp(HttpMethod):
    def __init__(self, conkey="", seckey=""):
        self.API_CONKEY = conkey.encode('utf-8')
        self.API_SECRET = seckey.encode('utf-8')
        super(BithumbHttp, self).__init__()

    @property
    def base_url(self):
        return "https://api.bithumb.com"

class PublicApi:
    @staticmethod
    def ticker(is_deatils = False):
        uri = f"/v1/market/all?isDetails={is_deatils}"
        return BithumbHttp().get(uri)
    
class Bithumb:
    @staticmethod
    def get_tickers(is_deatils=False):
        """
        빗썸이 지원하는 KRW 마켓 코드 30개
        :param payment_currency : KRW
        :return:
        """
        resp = None
        try:
            resp = PublicApi.ticker(is_deatils)

            krw_ticker = [item["market"] for item in resp if item["market"].startswith("KRW-")]
            krw_ticker = krw_ticker[:30]
            return krw_ticker
        except Exception:
            return resp

In [58]:
len(Bithumb.get_tickers())

30

In [48]:
len(krw_ticker)

377